# Model Checkpointing

In [1]:
import nltk
from sklearn.model_selection import train_test_split

import torch
from xtagger import LSTMForTagging
from xtagger import xtagger_dataset_to_df, df_to_torchtext_data


nltk_data = list(nltk.corpus.conll2000.tagged_sents(tagset='universal'))
train_set, test_set = train_test_split(nltk_data,train_size=0.8,test_size=0.2)

df_train = xtagger_dataset_to_df(train_set)
df_test = xtagger_dataset_to_df(test_set)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_iterator, valid_iterator, test_iterator, TEXT, TAGS = df_to_torchtext_data(
    df_train, 
    df_test, 
    device, 
    batch_size = 32
)

Number of training examples: 8758
Number of testing examples: 2190
Unique tokens in TEXT vocabulary: 17491
Unique tokens in TAGS vocabulary: 13


In [2]:
input_dim = len(TEXT.vocab)
out_dim = len(TAGS.vocab)
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]
tag_pad_idx = TAGS.vocab.stoi[TAGS.pad_token]


model = LSTMForTagging(
    input_dim, 
    out_dim, 
    TEXT, 
    TAGS, 
    cuda=True
)

The model has 2,383,225 trainable parameters


In [3]:
from xtagger import Checkpointing

checkpointing = Checkpointing(
    model_path = "./", 
    model_name = "lstm_tagger.pt", 
    monitor = "eval_acc",
    mode = "maximize",
    verbose = 1
)

In [4]:
model.fit(
    train_iterator,
    test_iterator, 
    epochs = 3,
    eval_metrics=["acc", "avg_f1"],
    checkpointing = checkpointing
)

Evaluating...



Model is saved with eval_acc = 94.85701151186623
{'eval': {'acc': 94.85701151186623, 'avg_f1': {'weighted': 94.82745513314725, 'micro': 94.85701151186623, 'macro': 80.37216870819536}}, 'train': {'acc': 96.05600854531482, 'avg_f1': {'weighted': 96.02388608634264, 'micro': 96.05600854531482, 'macro': 81.20326678273354}}, 'eval_loss': 0.1590846738089686, 'train_loss': 0.12938480307586}
Evaluating...



Model is saved with eval_acc = 96.49472292987531
{'eval': {'acc': 96.49472292987531, 'avg_f1': {'weighted': 96.4709498115255, 'micro': 96.49472292987531, 'macro': 81.70669100607397}}, 'train': {'acc': 98.34652024920612, 'avg_f1': {'weighted': 98.32902543494357, 'micro': 98.34652024920612, 'macro': 83.1959257568548}}, 'eval_loss': 0.1038704178687455, 'train_loss': 0.05422311709908238}
Evaluating...



Model is saved with eval_acc = 96.8260961173789
{'eval': {'acc': 96.8260961173789, 'avg_f1': {'weighted': 96.82928259419015, 'micro': 96.8260961173789, 'macro': 82.06780272336617}}, 'train': {'acc': 99.08795197610405, 'avg_f1': {'weighted': 99.0761306178256, 'micro': 99.08795197610405, 'macro': 83.83178471296974}}, 'eval_loss': 0.09836322031375291, 'train_loss': 0.029670139710099377}



In [5]:
model = LSTMForTagging(input_dim, out_dim, TEXT, TAGS, cuda=True)
model = checkpointing.load(model)
model.evaluate(valid_iterator)

The model has 2,383,225 trainable parameters


{'acc': 96.82603532161053}